In [4]:
import copy
import pickle
import numpy as NP
# from scipy.spatial import KDTree
import scipy as SP
# from scipy import sparse
from astropy.io import ascii
from astropy import units as U
from astropy import constants as FC

from IPython.core.debugger import set_trace

In [5]:
def ix_to_unraveled_ind(ixlist):
    ind_list = [(NP.repeat(ix_[0], ix_[1].size, axis=1).ravel(), NP.repeat(ix_[1], ix_[0].size, axis=0).ravel()) for ix_ in ixlist]
    return ind_list

def unraveled_to_raveled(unraveled_ind_list, shape):
    ind_list = [NP.ravel_multi_index(ind, shape) for ind in unraveled_ind_list]
    return ind_list

def rect2DNN_to_ix_(ycornerNN, xcornerNN, ysizeNN, xsizeNN):
    ycornerNN = NP.array(ycornerNN)
    xcornerNN = NP.array(xcornerNN)
    ysizeNN = NP.array(ysizeNN)
    xsizeNN = NP.array(xsizeNN)
    ix_ = [NP.ix_(ycornerNN[i]+NP.arange(ysizeNN[i]), xcornerNN[i]+NP.arange(xsizeNN[i])) if xsizeNN[i]*ysizeNN[i] > 0 else NP.ix_([], []) for i in range(len(xcornerNN))]
    return ix_

# def rect2DNN_old(x_ngbrof, y_ngbrof, x_ngbrin, y_ngbrin, delX=1, delY=1):
#     xofkdt = KDTree(x_ngbrof.reshape(-1,1))
#     yofkdt = KDTree(y_ngbrof.reshape(-1,1))
#     xinkdt = KDTree(x_ngbrin.reshape(-1,1))
#     yinkdt = KDTree(y_ngbrin.reshape(-1,1))
#     indxNN_list = xofkdt.query_ball_tree(xinkdt, delX, p=1)
#     indyNN_list = yofkdt.query_ball_tree(yinkdt, delY, p=1)
#     xsizeNN = [len(indxNN_list[i]) for i in range(x_ngbrof.size)]
#     ysizeNN = [len(indyNN_list[i]) for i in range(y_ngbrof.size)]
#     areaNN = NP.array(xsizeNN) * NP.array(ysizeNN)
#     xcornerNN = [NP.min(indxNN_list[i]) if xsizeNN[i]>0 else None for i in range(x_ngbrof.size)]
#     ycornerNN = [NP.min(indyNN_list[i]) if ysizeNN[i]>0 else None for i in range(y_ngbrof.size)]
#     return (list(zip(ycornerNN, xcornerNN)), list(zip(ysizeNN, xsizeNN)), areaNN)

def rect2DNN(x_ngbrof, y_ngbrof, x_ngbrin, y_ngbrin, delX=1, delY=1):
    dx = x_ngbrin[1] - x_ngbrin[0]
    dy = y_ngbrin[1] - y_ngbrin[0]
    delX_fltind = delX / dx
    delY_fltind = delY / dy
    x_ngbrof_fltind = (x_ngbrof - x_ngbrin.min()) / dx
    y_ngbrof_fltind = (y_ngbrof - y_ngbrin.min()) / dy
    delX_rounded = NP.round(delX_fltind).astype(int)
    delY_rounded = NP.round(delY_fltind).astype(int)
    xsizeNN = NP.clip(delX_rounded, 1, None).astype(int)
    ysizeNN = NP.clip(delY_rounded, 1, None).astype(int)
    xcornerNN = NP.round(x_ngbrof_fltind).astype(int) - xsizeNN//2
    ycornerNN = NP.round(y_ngbrof_fltind).astype(int) - ysizeNN//2
    n_oob_xleft = NP.clip(-xcornerNN, 0, None).astype(int)
    n_oob_xright = NP.clip(xcornerNN-(x_ngbrin.size-1), 0, None).astype(int)
    n_oob_yleft = NP.clip(-ycornerNN, 0, None).astype(int)
    n_oob_yright = NP.clip(ycornerNN-(y_ngbrin.size-1), 0, None).astype(int)
    xsizeNN -= (n_oob_xleft + n_oob_xright)
    ysizeNN -= (n_oob_yleft + n_oob_yright)
    areaNN = NP.array(xsizeNN) * NP.array(ysizeNN)
    xcornerNN = NP.clip(xcornerNN, 0, x_ngbrin.size-1).astype(int)
    ycornerNN = NP.clip(ycornerNN, 0, y_ngbrin.size-1).astype(int)
    return (xcornerNN, ycornerNN, xsizeNN, ysizeNN, areaNN)

# def gaussian2D_kernel(x, y, sigma=1.0):
#     return NP.exp(-(x**2 + y**2) / (2 * sigma**2))

# def rect2D_kernel(x, y, delX=1, delY=1):
#     kernel = NP.zeros(x.shape, dtype=float)
#     select_ind = NP.where(NP.logical_and(NP.abs(x)<=delX, NP.abs(y)<=delY))
#     kernel[select_ind] = 1.0
#     return kernel

In [6]:
def grid(rangelist, pad=None, spacing=None, pow2=True, verbose=True):

    """
    -----------------------------------------------------------------------------
    Produce a multi-dimensional grid.
    
    Inputs:
    rangelist [list of tuples] Each tuple is made of two elements, the min and
              max with min < max. One tuple for each axis.
             
    pad       [Optional. Scalar or list] The padding (in same units as range) to
              be applied along the axes. Default=None implies no padding.
              
    spacing   [Optional. Scalar or list] The spacing for the grid along each of
              the axes. If not supplied, a default of sqrt(max-min) is used. If a
              scalar is supplied, it applies for all axes. A list applies for
              each of the axes.
              
    pow2      [Optional, default=True] If set, the grid produced is a power of 2
              in all axes, which is useful for FFT.
              
    verbose   [Default=True]

    Outputs:

    tupleout  [List of tuples] A 4-element tuple for each axis. The elements in
              each tuple are min, max, lengths, and spacing (which could have
              been modified relative to input). The precise grid points can be
              generated by using numpy's linspace using min, max and lengths.
    -----------------------------------------------------------------------------
    """

    for item in rangelist:
        if item[0] >= item[1]:
            raise ValueError('Data ranges provided not compatible with min < max. Exiting from grid().')

    if pad is None:
        pad = NP.zeros(len(rangelist))
    elif isinstance(pad, (int,float)):
        pad = [pad]
    elif isinstance(pad, NP.ndarray):
        pad = pad.tolist()
    elif not isinstance(pad, list):
        raise TypeError('pad must be set to None, scalar, list or numpy array')
        
    if len(pad) == 1:
        pad = [pad] * len(rangelist)
    elif len(pad) > len(rangelist):
        pad = NP.asarray(pad[:len(rangelist)])
    elif (len(pad) > 1) and (len(pad) < len(rangelist)):
        if verbose is True:
            print('Insufficient paddings provided compared to the number of data ranges.')
            print('Assuming the remaining paddings to be zero.')
        pad += [0.0 for ranges in rangelist[len(pad):]]
    pad = NP.asarray(pad).flatten()
    pad.clip(min(NP.abs(pad))) # Remove any negative values for pad

    if spacing is None:
        if verbose is True:
            print('No spacing provided. Setting defaults to sqrt(max-min)')
            print('Final spacings could be different from defaults assumed.')
        spacing = [NP.sqrt(rangelist[i][1]-rangelist[i][0]+2*pad[i]) for i in range(len(rangelist))]
    elif isinstance(spacing, (int, float)):
        if verbose is True:
            print('Scalar value for spacing provided. Assuming spacing is identical along all axes.')
        spacing = [spacing] * len(rangelist)
    elif len(spacing) > len(rangelist):
        if verbose is True:
            print('Too many values of spacing provided. Ignoring values for indices beyond the length of data ranges.')
        spacing = NP.asarray(spacing[:len(rangelist)])
    elif (len(spacing) > 1) and (len(spacing) < len(rangelist)):
        if verbose is True:
            print('Insufficient spacings provided compared to the number of data ranges.')
            print('Assuming the remaining spacings to be default spacing.')
            print('Final spacings could be different from defaults assumed.')
        # spacing += [NP.sqrt(ranges[1]-ranges[0]) for ranges in rangelist[len(spacing):]]
        spacing += [NP.sqrt(rangelist[i][1]-rangelist[i][0]+2*pad[i]) for i in range(len(spacing),len(rangelist))]
    spacing = NP.asarray(spacing).flatten()
    spacing.clip(min(NP.abs(spacing)))

    rangelist = NP.asarray(rangelist)
    lengths = NP.ceil((rangelist[:,1]-rangelist[:,0]+2*pad)/spacing)+1
    lengths = lengths.astype(int)

    for i in range(len(lengths)): 
        if (lengths[i] % 2) == 0: lengths[i] += 1
        # make it odd number of bin edges enclsoing first
        # and last intervals so that the mid-point is one
        # of the bin edges.

    if pow2 is True:
        lengths = 2**NP.ceil(NP.log2(lengths))
        lengths = lengths.astype(int)
        newspacing = (rangelist[:,1]-rangelist[:,0]+2*pad)/(lengths-2)
        offsets = rangelist[:,0]-pad+(lengths-2)*newspacing - (rangelist[:,1]+pad)
        tupleout = list(zip(rangelist[:,0]-pad-0.5*offsets-newspacing, rangelist[:,1]+pad+0.5*offsets, lengths, newspacing)) # converts numpy arrays into a list of tuples
    else:
        offsets = rangelist[:,0]-pad+(lengths-1)*spacing - (rangelist[:,1]+pad)
        tupleout = list(zip(rangelist[:,0]-pad-0.5*offsets, rangelist[:,1]+pad+0.5*offsets, lengths, spacing)) # converts numpy arrays into a list of tuples
    
    return tupleout


In [7]:
def gen_ant_grid_wts_cube(vusize, raveled_ind_lol, ant_grid_wts_lol):
    nchan = len(raveled_ind_lol)
    npol = 2
    outarr = NP.zeros((nchan,npol,npol,vusize), dtype=complex) # (nchan,npol,npol,nvusize)
    for iwl in range(nchan):
        for ravind, awts in zip(raveled_ind_lol[iwl], ant_grid_wts_lol[iwl]):
            ixind = NP.ix_([iwl], NP.arange(npol), NP.arange(npol), ravind)
            outarr[ixind] += awts[NP.newaxis,...] # (1,npol,npol,npix)
    return outarr

def gen_grid_Ef_cube(vusize, raveled_ind_lol, grid_Ef_chan_list):
    nchan = len(raveled_ind_lol)
    nruns = grid_Ef_chan_list[0].shape[0]
    nant = len(raveled_ind_lol[0])
    npol = 2
    outunit = grid_Ef_chan_list[0].unit
    outarr = NP.zeros((nruns,nchan,npol,vusize), dtype=complex) * outunit # (nruns, nchan, npol, nvusize)
    for iwl in range(nchan):
        for ii,ravind in enumerate(raveled_ind_lol[iwl]):
            ixind = NP.ix_(NP.arange(nruns), [iwl], NP.arange(npol), ravind)
            outarr[ixind] += grid_Ef_chan_list[iwl][:,ii,:,:][:,NP.newaxis,...] # (nruns,1,npol,npix)
    return outarr

def gen_image(inparr, fftshape=None, fftaxes=(-2,-1), polaxis=-4, freqaxis=-3, bwsyn=False):
    polaxis = (polaxis+inparr.ndim) % inparr.ndim
    freqaxis = (freqaxis+inparr.ndim) % inparr.ndim
    outarr = SP.fft.fft2(inparr, s=fftshape, axes=fftaxes)
    outarr = NP.expand_dims(outarr, axis=polaxis) * NP.expand_dims(outarr, axis=polaxis+1).conj()
    if bwsyn:
        outarr = NP.sum(outarr, axis=freqaxis, keepdims=True)
    outarr = SP.fft.fftshift(outarr, axes=fftaxes)
    return outarr

In [8]:
def stochastic_E_spectrum(freq_center, nchan, channel_width, flux_ref=1.0,
                          freq_ref=None, spectral_index=0.0, skypos=None, 
                          ref_point=None, antpos=[0.0,0.0,0.0], nruns=1, 
                          voltage_pattern=None, verbose=True):

    """
    ----------------------------------------------------------------------------
    Compute a stochastic electric field spectrum obtained from sources with 
    given flux densities and spectral indices at given positions at specified 
    antenna locations. 

    Inputs:

    freq_center      [float] Center frequency in Hz. Center frequency must be
                     greater than half the bandwidth.

    nchan            [integer] Number of frequency channels in spectrum

    channel_width    [float] Channel width 

    Keyword Inputs:

    flux_ref         [list or numpy array of float] Flux densities of sources
                     at the respective reference frequencies. Units are 
                     arbitrary. Values have to be positive. Default = 1.0. 

    freq_ref         [list or numpy array of float] Reference frequency (Hz). 
                     If not provided, default is set to center frequency given
                     in freq_center for each of the sources. If a single value 
                     is provided, it will be applicable to all the sources. If a 
                     list or numpy array is provided, it should be of size equal
                     to that of flux_ref. 

    spectral_index   [list or numpy array of float] Spectral Index 
                     (flux ~ freq ** alpha). If not provided, default is set to 
                     zero, a flat spectrum, for each of the sources. If a single 
                     value is provided, it will be applicable to all the sources. 
                     If a list or numpy array is provided, it should be of size 
                     equal to that of flux_ref. 

    skypos           [list, tuple, list of lists, list of tuples, numpy array]
                     Sky positions of sources provided in direction cosine
                     coordinates aligned with local ENU axes. It should be a
                     3-element list, a 3-element tuple, a list of 3-element
                     lists, list of 3-element tuples, or a 3-column numpy array.
                     Each 3-element entity corresponds to a source position. 
                     Number of 3-element entities should equal the number of 
                     sources as specified by the size of flux_ref. Rules of 
                     direction cosine quantities should be followed. If only 
                     one  source is specified by flux_ref and skypos is not
                     specified, skypos defaults to the zenith (0.0, 0.0, 1.0)

    ref_point        [3-element list, tuple, or numpy vector] Point on sky used
                     as a phase reference. Same units as skypos (which is
                     direction cosines and must satisfy rules of direction
                     cosines). If None provided, it defaults to zenith
                     (0.0, 0.0, 1.0)

    antpos           [list, tuple, list of lists, list of tuples, numpy array]
                     Antenna positions provided along local ENU axes. 
                     It should be a 3-element list, a 3-element tuple, a list of 
                     3-element lists, list of 3-element tuples, or a 3-column 
                     numpy array. Each 3-element entity corresponds to an
                     antenna position. If not specified, antpos by default is 
                     assigned the origin (0.0, 0.0, 0.0).

    voltage_pattern  [numpy array] Voltage pattern for each frequency channel
                     at each source location for each antenna. It must be of
                     shape nsrc x nchan x nant. If any of these dimensions are
                     1, it is assumed to be identical along that direction. 
                     If specified as None (default), it is assumed to be unity
                     and identical across antennas, sky locations and frequency
                     channels. 

    verbose:         [boolean] If set to True, prints progress and diagnostic
                     messages. Default = True.
    
    Output:

    dictout          [dictionary] Consists of the following tags and info:
                     'f'        [numpy array] frequencies of the channels in the 
                                spectrum of size nchan
                     'Ef'       [complex numpy array] (nruns, nant, nchan, npol)  
                                numpy array consisting of complex stochastic 
                                electric field spectra. nchan is the number of  
                                channels in the spectrum and nant is the number 
                                of antennas.
                     'antpos'   [numpy array] 3-column array of antenna
                                positions (same as the input argument antpos)

    ----------------------------------------------------------------------------
    """

    nsrc = flux_ref.size
    nant = antpos.shape[0]
    npol = 2
    if voltage_pattern is None:
        voltage_pattern = NP.diag(NP.ones(2))[NP.newaxis,NP.newaxis,NP.newaxis,NP.newaxis,...]
    vb_shape = voltage_pattern.shape

    freqs = freq_center + channel_width * (NP.arange(nchan) - nchan//2)
    if spectral_index is None:
        spectral_index = NP.zeros((1,1,1,1,1))
    alpha = spectral_index.reshape(-1,1,1,1,1)
    freqs = freqs.reshape(1,1,1,-1,1)
    freq_ratio = freqs / freq_ref
    flux_spectra = flux_ref.reshape(nsrc,1,1,1,1) * (freq_ratio ** alpha)
    sigmas = NP.sqrt(flux_spectra)
    Ef_amp = sigmas/NP.sqrt(2) * (NP.random.normal(loc=0.0, scale=1.0, size=(nsrc,nruns,1,nchan,npol)) + 1j * NP.random.normal(loc=0.0, scale=1.0, size=(nsrc,nruns,1,nchan,npol)))
    Ef_phase = 1.0
    Ef_sky = Ef_amp * Ef_phase

    skypos_dot_antpos = NP.dot(skypos-ref_point, antpos.T) # (nsrc, nant)
    k_dot_r_phase = 2.0 * NP.pi * freqs / FC.c * skypos_dot_antpos[:,NP.newaxis,:,NP.newaxis,NP.newaxis] # (nsrc, nant, nchan)
    Ef_ant = NP.einsum('i...mn,i...n->...m', voltage_pattern, Ef_sky * NP.exp(1j * k_dot_r_phase)) # (nruns, nant, nchan, npol)
    if verbose:
        print('Performed linear superposition of electric fields from source(s).')
    dictout = {}
    dictout['f'] = freqs.ravel()
    dictout['Ef'] = Ef_ant # (nruns, nant, nchan, npol)
    dictout['antpos'] = antpos

    if verbose:
        print('stochastic_E_spectrum() executed successfully.\n')

    return dictout


In [37]:
def minmax_scaler(inp, low=0.0, high=1.0, axis=None):
    """
    ----------------------------------------------------------------------------
    Scale an input between minimum and maximum values linearly

    Inputs:

    inp  [numpy array] An N-dimensional numpy array which is to be scaled

    low  [scalar] Minimum value of inp will be scaled to this value

    high [scalar] Maximum value of inp will be scaled to this value

    axis [scalar or None] If set, the scaling occurs differently for each 
         element along that axis. If set to None (default), a flattened array is
         used to determine the max and min values for the entire array

    Output:

    Numpy array after scaling
    ----------------------------------------------------------------------------
    """
    max_absreal = NP.max(NP.abs(inp.real), axis=axis, keepdims=True)
    if NP.iscomplexobj(inp):    
        max_absimag = NP.max(NP.abs(inp.imag), axis=axis, keepdims=True)
        high_in = NP.max(NP.concatenate([max_absreal[NP.newaxis,...], max_absimag[NP.newaxis,...]], axis=0), axis=0)
    else:
        high_in = max_absreal
    low_in = -high_in
    inp_norm = (inp - (-high_in)) / (high_in - (-high_in))
    inp_scaled = inp_norm * (high - low) + low
    return inp_scaled

def quantize(inparr, nbits=8, method='round', axis=None):
    nlevs = 2**nbits 
    outarr = minmax_scaler(inparr, low=0.0, high=1.0, axis=axis)
    outarr *= (nlevs-1)
    if method.lower() == 'round':
        outarr = NP.round(outarr)
    outarr = outarr.astype('uint{0}'.format(nbits))
    return outarr

In [49]:
nbits = 8
nlevels = 2**nbits
aa = NP.random.randn(40).reshape(5,8) * 16
bb = NP.random.randn(40).reshape(5,8) * 16
cc = aa + 1j * bb
dd = NP.concatenate([NP.max(aa, axis=0, keepdims=True), NP.max(bb, axis=0, keepdims=True)], axis=0)
mmsaa = minmax_scaler(aa)
saa = mmsaa * (nlevels-1)
qsaa = NP.round(saa).astype('uint{0}'.format(nbits))
print(saa.min())
print(saa.max())
print(qsaa.min())
print(qsaa.max())
print(qsaa.dtype)
print(dd.shape)
print(NP.min(dd, axis=0, keepdims=True).shape)
print(mmsaa.min(), mmsaa.max())


15.185707857853982
255.0
15
255
uint8
(2, 8)
(1, 8)
0.05955179552099601 1.0


In [6]:
# def nearest_nghbrs(input_locations, grid_x, grid_y, distUL, dist_norm=2.0, workers=1):
#     # Create a grid of 2D x and y locations
#     grid_points = NP.column_stack((grid_x.flatten(), grid_y.flatten()))

#     # Build KDTree using grid locations
#     kdt = KDTree(grid_points)
#     list_indNN = kdt.query_ball_point(input_locations, distUL, p=dist_norm, 
#                                       eps=0, workers=1, return_sorted=None, 
#                                       return_length=False)
#     return list_indNN

# def gridder(input_locations, input_values, grid_x, grid_y, kernel_func, dist_norm=2.0, workers=1):
#     list_indNN = nearest_nghbrs(input_locations, grid_x, grid_y, dist_norm=dist_norm, workers=workers)
#     result_grid = NP.zeros_like(grid_x, dtype=float)
#     for input_loc, input_val in zip(input_locations, input_values):
#         # Find nearest grid locations using KDTree
#         if distance_metric == 'l2':
#             _, nearest_indices = kdt.query(input_loc, k=4)  # You can adjust 'k' based on your requirements
#         elif distance_metric == 'manhattan':
#             _, nearest_indices = kdt.query(input_loc, k=4, p=1)
#         else:
#             raise ValueError("Invalid distance metric. Use 'l2' or 'manhattan'.")

#         # Calculate distances and kernel weights
#         distances = NP.linalg.norm(grid_points[nearest_indices] - input_loc, axis=1)
#         weights = kernel_func(*NP.transpose(distances))

#         # Normalize weights
#         weights /= NP.sum(weights)

#         # Extrapolate input value onto grid locations
#         result_grid += NP.sum(weights[:, NP.newaxis, NP.newaxis] * input_val, axis=0).reshape(result_grid.shape)

#     return result_grid

In [40]:
fc = 100*U.MHz
df = 100 * U.kHz
nchan = 10
duration = 100*U.ms
bw = nchan * df
freqs = fc + df * (NP.arange(nchan) - nchan//2)
wl = (FC.c / freqs).to(U.m)
delT = (1/df).to(U.s)
dt = (1/bw).to(U.s)
nruns = NP.round((duration/delT).decompose()).astype(int)

In [41]:
datadir = '/mnt/data/users/thy009/projects/multi-scale-imaging-architectures/data/'
aavs_antlocs_file = 'AAVS2_loc_italia_190429.txt'
anttab = ascii.read(datadir+aavs_antlocs_file, names=['id', 'x', 'y', 'z'], format='no_header')
anttab['id'] = anttab['id'].astype(int)
anttab['x'] = anttab['x'] * U.m
anttab['y'] = anttab['y'] * U.m
anttab['z'] = anttab['z'] * U.m
nants = anttab['id'].size
antx = anttab['x']
anty = anttab['y']
antz = anttab['z']
antx = antx - NP.mean(antx) * anttab['x'].unit
anty = anty - NP.mean(anty) * anttab['y'].unit
antz = antz - NP.mean(antz) * anttab['z'].unit

In [42]:
anttab.pprint()

 id   x      y    z  
      m      m    m  
--- ------ ----- ----
  1  17.47 -6.81 15.0
  2   17.7 -3.94 16.0
  3  18.89 -1.21 16.0
  4  18.11  1.02 16.0
  5  17.92  3.28 16.0
  6  15.75  8.85  5.0
  7  15.85  6.68  5.0
  8  16.94  5.18  5.0
  9  15.17 -0.69 16.0
 10  16.19 -1.99 16.0
...    ...   ...  ...
247 -16.08  4.29  9.0
248 -17.09  5.57  9.0
249  -15.4  6.96  9.0
250 -16.04 10.18  9.0
251 -17.32  7.53  9.0
252 -18.67  3.36  9.0
253 -18.51  1.41 10.0
254 -18.26 -0.33 10.0
255 -17.35 -3.38 10.0
256  -17.6 -6.37 11.0
Length = 256 rows


In [43]:
au = antx.reshape(-1,1) / wl.reshape(1,-1)
av = anty.reshape(-1,1) / wl.reshape(1,-1)
aw = antz.reshape(-1,1) / wl.reshape(1,-1)
au_min = au.min()
av_min = av.min()
aw_min = aw.min()
au_max = au.max()
av_max = av.max()
aw_max = aw.max()
print(au_min, av_min, aw_min)
print(au_max, av_max, aw_max)

-6.135834915666892 -6.3076665503539795 -2.511737636842085
6.442947169638272 6.307954353624867 2.511737636842085


In [44]:
print(anttab['z'].min(), anttab['z'].max())
print(NP.mean(anttab['x']))
print(NP.mean(anttab['y']))
print(NP.mean(anttab['z']))
print(antx.shape)

1.0 16.0
-0.34851562499999966
-0.08542968750000002
8.5
(256,)


In [45]:
print(au.min(), au.max())
print(av.min(), av.max())

-6.135834915666892 6.442947169638272
-6.3076665503539795 6.307954353624867


In [46]:
sizex_ant = 2.0 * U.m
sizey_ant = 2.0 * U.m 
upad = sizex_ant / wl.min()
vpad = sizey_ant / wl.min()
du_ant = sizex_ant / wl.max()
dv_ant = sizey_ant / wl.max()
ant_uvec_info, ant_vvec_info = grid([(au_min,au_max), (av_min,av_max)], pad=[upad, vpad], spacing=[du_ant, dv_ant], pow2=True)
ant_uvec = NP.linspace(ant_uvec_info[0], ant_uvec_info[1], num=ant_uvec_info[2], endpoint=True)
ant_vvec = NP.linspace(ant_vvec_info[0], ant_vvec_info[1], num=ant_vvec_info[2], endpoint=True)
ant_ugrid, ant_vgrid = NP.meshgrid(ant_uvec, ant_vvec)
print(ant_ugrid.shape)


(32, 32)


In [47]:
nsrc = 4
flux_ref = (1.0+NP.arange(nsrc))*U.Jy
skypos = NP.array([[0.0, 0.0, 1.0], 
                   [0.5, 0.0, NP.sqrt(3.0)/2],
                   [0.0, -0.5, NP.sqrt(3.0)/2],
                   [-0.5, 0.5, NP.sqrt(1.0)/2],
                   ])
ref_point= NP.array([[0.0, 0.0, 1.0]])
antpos = NP.hstack([antx.reshape(-1,1), anty.reshape(-1,1), antz.reshape(-1,1)])
ant_E_dynspec = stochastic_E_spectrum(fc, nchan, df, flux_ref, freq_ref=fc, 
                                      spectral_index=None, skypos=skypos, 
                                      ref_point=ref_point, antpos=antpos, 
                                      nruns=nruns, voltage_pattern=None)


Performed linear superposition of electric fields from source(s).
stochastic_E_spectrum() executed successfully.



In [48]:
# if True:
#     set_trace()
#     quant_ant_Etafp = quantize(ant_E_dynspec['Ef'].to('Jy(1/2)').value)

> /tmp/ipykernel_103576/4160575896.py(3)<module>()
      1 if True:
      2     set_trace()
----> 3     quant_ant_Etafp = quantize(ant_E_dynspec['Ef'].to('Jy(1/2)').value)

--Call--
> /mnt/data/users/thy009/software/installed/miniforge3/envs/utils/lib/python3.12/site-packages/astropy/units/quantity.py(905)to()
    903             return result
    904 
--> 905     def to(self, unit, equivalencies=[], copy=True):
    906         """
    907         Return a new `~astropy.units.Quantity` object with the specified unit.

--Return--
<Quantity [[[...j]]]] Jy(1/2)>
> /mnt/data/users/thy009/software/installed/miniforge3/envs/utils/lib/python3.12/site-packages/astropy/units/quantity.py(942)to()
    940             # to_value only copies if necessary
    941             value = self.to_value(unit, equivalencies)
--> 942         return self._new_view(value, unit)
    943 
    944     def to_value(self, unit=None, equivalencies=[]):

--Call--
> /mnt/data/users/thy009/software/installed/miniforge3

In [15]:
ix_list = []
unraveled_ind_lol = []
raveled_ind_lol = []
ant_grid_wts_lol = []
delX = (NP.zeros((nants,1))+sizex_ant/wl.reshape(1,-1)).ravel()
delY = (NP.zeros((nants,1))+sizey_ant/wl.reshape(1,-1)).ravel()
xcornerNN, ycornerNN, xsizeNN, ysizeNN, areaNN = rect2DNN(au.ravel(), av.ravel(), ant_uvec, ant_vvec, delX=delX, delY=delY)
xcornerNN = xcornerNN.reshape(nants, -1)
ycornerNN = ycornerNN.reshape(nants, -1)
xsizeNN = xsizeNN.reshape(nants, -1)
ysizeNN = ysizeNN.reshape(nants, -1)
areaNN = areaNN.reshape(nants, -1)
for iwl in range(wl.size):    
    ix_ = rect2DNN_to_ix_(ycornerNN[:,iwl], xcornerNN[:,iwl], ysizeNN[:,iwl], xsizeNN[:,iwl])
    ix_list += [copy.deepcopy(ix_)]
    unraveled_ind = ix_to_unraveled_ind(ix_)
    unraveled_ind_lol += [copy.deepcopy(unraveled_ind)]
    raveled_ind = unraveled_to_raveled(unraveled_ind, ant_ugrid.shape)
    raveled_ind_lol += [copy.deepcopy(raveled_ind)]
    ant_grid_wts = [NP.diag(NP.ones(2))[...,NP.newaxis]*NP.ones((1,1,len(item))) for item in raveled_ind]
    ant_grid_wts_lol += [copy.deepcopy(ant_grid_wts)]


In [16]:
NP.savez_compressed(datadir+'epic-test-data-{0:0d}-runs-{1:0d}x{2:0d}-grid-{3:0d}-ants-{4:0d}-chans.npz'.format(nruns, ant_ugrid.shape[0], ant_ugrid.shape[1], nants, nchan), ant_Etafp=ant_E_dynspec['Ef'].to('Jy(1/2)').value, antpos=NP.hstack([antx.reshape(-1,1),anty.reshape(-1,1),antz.reshape(-1,1)]), freqs=freqs.to('MHz').value, xcorners=xcornerNN, ycorners=ycornerNN, xsizes=xsizeNN, ysizes=ysizeNN)

In [17]:
with open(datadir+'epic-test-ant-wts-{0:0d}x{1:0d}-grid-{2:0d}-ants-{3:0d}-chans.pkl'.format(ant_ugrid.shape[0], ant_ugrid.shape[1], nants, nchan), 'wb') as fobj:
    pickle.dump(ant_grid_wts_lol, fobj)

In [18]:
npol = 2
nri = 2
nbytes_per_sample = 1
print(nchan*nruns*nants*npol*nri*nbytes_per_sample/2**30) # in GB

0.095367431640625


In [19]:
# distUL = 3.0
# xv = NP.arange(12)
# yv = NP.arange(16)
# xg, yg = NP.meshgrid(xv, yv)
# xlocs = NP.array([2.5, 5.2, 8.6])
# ylocs = NP.array([3.5, 9.2, 6.9])
# xgrel = xg.ravel()[NP.newaxis,...] - xlocs[:,NP.newaxis]
# ygrel = yg.ravel()[NP.newaxis,...] - ylocs[:,NP.newaxis]
# layered_rectkern = rect2D_kernel(xgrel, ygrel, delX=2, delY=2)
# # layered_rectkern = gaussian2D_kernel(xgrel, ygrel, sigma=1)   
# layered_rectkern = layered_rectkern.reshape(xlocs.shape+xg.shape) 
# valgrid = NP.sum(NP.ones(xlocs.shape, dtype=float).reshape(-1,1,1) * layered_rectkern, axis=0)
# print(valgrid)

In [20]:
# fig = PLT.figure(figsize=(4,4))
# ax = fig.add_subplot(111)
# ax.imshow(valgrid)

In [21]:
# indNN_L1_raveled = nearest_nghbrs(NP.column_stack([xlocs, ylocs]), xg, yg, distUL, dist_norm=1)
# indNN_L2_raveled = nearest_nghbrs(NP.column_stack([xlocs, ylocs]), xg, yg, distUL, dist_norm=2)
# print(indNN_L2_raveled)

In [22]:
# indNN_L1 = [NP.unravel_index(indNN_L1_raveled[i], xg.shape) for i in range(xlocs.size)]
# indNN_L2 = [NP.unravel_index(indNN_L2_raveled[i], xg.shape) for i in range(xlocs.size)]


In [23]:
# print(indNN_L1[0])
# print('-----------')
# print(indNN_L2)

In [24]:
# layered_L1_footprint = NP.zeros((xlocs.size,)+xg.shape, dtype=int)
# layered_L2_footprint = NP.zeros((xlocs.size,)+xg.shape, dtype=int)
# for i in range(xlocs.size):
#     fp1 = NP.zeros(xg.shape, dtype=int)
#     fp1[indNN_L1[i]] = 1
#     fp2 = NP.zeros(xg.shape, dtype=int)
#     fp2[indNN_L2[i]] = 1
#     layered_L1_footprint[i] = NP.copy(fp1)
#     layered_L2_footprint[i] = NP.copy(fp2)
# net_L1_footprint = NP.sum(layered_L1_footprint, axis=0)
# net_L2_footprint = NP.sum(layered_L2_footprint, axis=0)

In [25]:
# fig = PLT.figure(figsize=(4,4))
# ax = fig.add_subplot(111)
# ax.imshow(net_L2_footprint)

In [26]:
# print(indNN_L2_raveled)
# indNN_L2_raveled_new = LKP.find_NN(NP.column_stack([xlocs, ylocs]), NP.column_stack([xg.ravel(), yg.ravel()]), distance_ULIM=distUL)
# # print(indNN_L2_raveled_new[0])
# indNN_L2_new = [NP.unravel_index(indNN_L2_raveled_new[0][i], xg.shape) for i in range(xlocs.size)]